# Setup

In [53]:
### Libraries
import pandas as pd
from IPython.display import display
### Python OBDC bridge
import pyodbc 
### IRIS Python Native API
import irisnative

### SQL Connection parameters
dsn = 'IRIS IntegeratedML monitor'
server = 'irisimlsvr'
port = '51773'
database = 'USER' 
username = 'SUPERUSER' 
password = 'SYS' 
cnxn = pyodbc.connect('DRIVER={InterSystems ODBC35};SERVER='+server+';PORT='+port+';DATABASE='+database+';UID='+username+';PWD='+ password)

### charset stuffs...
cnxn.setdecoding(pyodbc.SQL_CHAR, encoding='utf8')
cnxn.setdecoding(pyodbc.SQL_WCHAR, encoding='utf8')
cnxn.setencoding(encoding='utf8')

### Get a SQL cursor
cursor = cnxn.cursor()

### Native API connection's parameters
ip = "irisimlsvr"
port = 51773
namespace = "USER"
username = "SUPERUSER"
password = "SYS"

### Create database connection and IRIS instance
try:
    nativeConn = irisnative.createConnection(ip, port, namespace, username, password)
    dbnative = irisnative.createIris(nativeConn)
except:
    print('Seems like you can\'t connect to IRIS... try to exit from any IRIS terminal or other license consumption session')

### Clean previous runs
df1 = pd.read_sql(
    "SELECT COUNT(*) AS ModelExists FROM INFORMATION_SCHEMA.ML_TRAINED_MODELS WHERE MODEL_NAME = 'AppointmentsPredection'", 
    cnxn
)
if (df1['ModelExists'][0] > 0):
    print('Deleting previous model...')
    cursor.execute("DROP MODEL AppointmentsPredection")
    print('Model deleted')
else:
    print('No previous runs found')

Error: ('HY000', '[HY000] [unixODBC][Iris ODBC][State : HY000][Native Code 417]\n[libirisodbcu35.so]\nAccess Denied (417) (SQLDriverConnect)')

# ML model setup

Let's create a model to predict appointments, using first 500 records to train it.

In [54]:
### Display dataset table
df1 = pd.read_sql("SELECT * FROM MedicalAppointments", cnxn)
display(df1)

,ID,Canal,CreacionDiaD,CreacionHoraD,CreacionMesD,Edad,Especialidad,Latencia,ReservaDiaD,ReservaHoraD,ReservaMesD,Sexo,Show,Tipo
0,1,1,4,14,12,66,76,18,1,18,1,1,False,1
1,2,1,5,11,12,66,76,17,1,19,1,1,False,1
2,3,3,2,10,12,35,18,20,1,15,1,1,False,1
3,4,1,2,8,11,44,51,34,1,19,1,2,False,1
4,5,1,5,10,12,72,76,17,1,12,1,2,False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61209,30657,3,1,12,1,4,23,45,4,14,3,1,True,1
61210,30658,1,5,16,1,2,39,34,4,8,3,1,True,1
61211,30659,1,4,10,1,43,51,35,4,9,3,1,True,1
61212,30660,3,7,18,1,38,18,46,4,8,3,1,True,1


In [55]:
### Create a model for predicting patient's appointments misses
### Note: seed parameter it's to force reproducibility
cursor.execute("CREATE MODEL AppointmentsPredection PREDICTING (Show) FROM MedicalAppointments USING {\"seed\": 3}")
print('Training model (this could take a while)...')
cursor.execute("TRAIN MODEL AppointmentsPredection FROM MedicalAppointments WHERE ID <= 500 USING {\"seed\": 3}")
print('Model trained')
cnxn.commit()

Error: ('HY000', "[HY000] [Iris ODBC][State : HY000][Native Code 180]\n[libirisodbcu35.so]\n[SQLCODE: <-180>:<Model name not unique>]\r\n[Location: <ServerLoop>]\r\n[%msg: <Model name 'AppointmentsPredection' is not unique>] (180) (SQLExecDirectW)")

In [56]:
### Display model information
cursor.execute("SELECT * FROM INFORMATION_SCHEMA.ML_TRAINED_MODELS")
df1 = pd.read_sql("SELECT * FROM INFORMATION_SCHEMA.ML_TRAINED_MODELS", cnxn)
display(df1)

,MODEL_NAME,TRAINED_MODEL_NAME,PROVIDER,TRAINED_TIMESTAMP,MODEL_TYPE,MODEL_INFO
0,AppointmentsPredection,AppointmentsPredection2,AutoML,2020-07-22 02:52:44.611,classification,"ModelType:Logistic Regression, Package:sklearn..."
1,AppointmentsPredection,AppointmentsPredection4,AutoML,2020-07-22 02:53:56.076,classification,"ModelType:TensorFlow Neural Network, Package:T..."


In [57]:
### Display prediction and real patient's appointments presence
df1 = pd.read_sql("SELECT PREDICT(AppointmentsPredection) As Predicted, Show FROM MedicalAppointments WHERE ID <= 500", cnxn)
display(df1)

,Predicted,Show
0,0,False
1,0,False
2,0,False
3,0,False
4,0,False
...,...,...
495,1,True
496,1,True
497,1,True
498,1,True


# Model monitoring

Now let's validate our model using 100 new records regargind train model, i.e., the 600 first records. Note that again the parameter seed was used in order to guarantee reproducibility.

In [58]:
### Model validation
cursor.execute("VALIDATE MODEL AppointmentsPredection FROM MedicalAppointments WHERE ID < 600 USING {\"seed\": 3}")
df5 = pd.read_sql("SELECT * FROM INFORMATION_SCHEMA.ML_VALIDATION_METRICS WHERE MODEL_NAME = 'AppointmentsPredection'", cnxn)
df6 = df5.pivot(index='VALIDATION_RUN_NAME', columns='METRIC_NAME', values='METRIC_VALUE')
display(df6)

METRIC_NAME,Accuracy,F-Measure,Precision,Recall
VALIDATION_RUN_NAME,,,,
AppointmentsPredection21,0.90,0.94,0.98,0.91
AppointmentsPredection22,0.87,0.93,0.98,0.88
AppointmentsPredection41,0.96,0.98,0.97,0.99
AppointmentsPredection42,0.96,0.98,0.97,0.99


As we can see, model's accuracy is about 90%.

MyMetric.IntegratedMLModelsValidation application monitor class let you to access such metrics. Thus it's possible to IRIS' ^%SYSMONMGR monitor utility being aware about your ML models perfomance.

In [61]:
### Get the last validation parameter for each IntegeratedML model
print(dbnative.classMethodValue("MyMetric.Install", "GetSamples", "MyMetric.IntegratedMLModelsValidation"))

MyMetric - IntegratedMLModelsValidation   AppointmentsPredection4 [2020-07-22 03:00:48]   Serial value of object or the object identity:WMyMetric||IntegratedMLModelsValidation||AppointmentsPredection4 [2020-07-22 03:00:48].MyMetric.Sample.IntegratedMLModelsValidation
   Accuracy:                   .96
   F-Measure:                  .98
   Precision:                  .97
   Recall:                     .99
   Name of the model definition:APPOINTMENTSPREDECTION
   Name of the trained model being validated:AppointmentsPredection4 [2020-07-22 03:00:48]
   Validation error (if encountered):1




Results could also be seen by CSP interface: http://localhost:8092/csp/user/MyMetric.Sample.IntegratedMLModelsValidation.cls

With such functionality, someone could setup an alert into ^%SYSMONMGR monitor utility, checking model's performance metrics against thresholds. This way, IntegeratedML applications performance issues could be quickly notified.

# Simulating a performance issue

Let's re-validate the model, but this turn, using first 800 records and see how this affects to model's performance metrics.

In [62]:
### Model validation
cursor.execute("VALIDATE MODEL AppointmentsPredection FROM MedicalAppointments WHERE ID < 800 USING {\"seed\": 3}")

In [63]:
### Get the last validation parameter for each IntegeratedML model
print(dbnative.classMethodValue("MyMetric.Install", "GetSamples", "MyMetric.IntegratedMLModelsValidation"))

MyMetric - IntegratedMLModelsValidation   AppointmentsPredection4 [2020-07-22 03:01:07]   Serial value of object or the object identity:WMyMetric||IntegratedMLModelsValidation||AppointmentsPredection4 [2020-07-22 03:01:07].MyMetric.Sample.IntegratedMLModelsValidation
   Accuracy:                   .96
   F-Measure:                  .98
   Precision:                  .97
   Recall:                     .99
   Name of the model definition:APPOINTMENTSPREDECTION
   Name of the trained model being validated:AppointmentsPredection4 [2020-07-22 03:01:07]
   Validation error (if encountered):1




Now model's accuracy had descreased to about 87%.  

Let's say that for you business rule, performances less than 90% are unacceptable. If you had previously setup an alert to notify your team when model's performance is less than 90%, such team could be quickly notified and take care about this issue.

In [51]:
### Retrain model using first 600 records
print('Training model (this could take a while)...')
cursor.execute("TRAIN MODEL AppointmentsPredection FROM MedicalAppointments WHERE ID <= 600 USING {\"seed\": 3}")
print('Model trained')
cnxn.commit()

### Model validation
cursor.execute("VALIDATE MODEL AppointmentsPredection FROM MedicalAppointments WHERE ID <= 800 USING {\"seed\": 3}")

Training model (this could take a while)...
Model trained


In [52]:
### Get the last validation parameter for each IntegeratedML model
print(dbnative.classMethodValue("MyMetric.Install", "GetSamples", "MyMetric.IntegratedMLModelsValidation"))

# Teardown

In [64]:
### Close the database
cnxn.close()

In [66]:
nativeConn.close()